In [1]:
from fastapi import FastAPI, Depends
from sqlalchemy.orm import Session

from database import get_db
from utils import init_db
from models import *
from api_functions import get_overview, get_chart_data, get_assets

import logging
logging.getLogger().setLevel(logging.INFO)

In [2]:
import pandas as pd
import requests
import datetime as dt
import time

from database import engine
from models import Base
import logging
logging.getLogger().setLevel(logging.INFO)

In [3]:
logging.info('Droping Existing Tables ...')
Base.metadata.drop_all(engine)

INFO:root:Droping Existing Tables ...


In [19]:
Base.metadata.tables.keys() == metadata.tables.keys()

True

In [18]:
metadata.tables.keys()

dict_keys(['BourseHistory', 'CryptocurrencyHistory', 'CurrencyHistory', 'GoldHistory', 'FundHistory'])

In [11]:
from sqlalchemy import MetaData
metadata = MetaData()
metadata.reflect(bind= engine)

In [16]:
metadata.tables

FacadeDict({'BourseHistory': Table('BourseHistory', MetaData(), Column('id', INTEGER(), table=<BourseHistory>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x7fdb6630ee50>, for_update=False)), Column('Index_Id', BIGINT(), table=<BourseHistory>, nullable=False), Column('Index_Type', VARCHAR(), table=<BourseHistory>, nullable=False), Column('Date_En', DATE(), table=<BourseHistory>, nullable=False), Column('Value', DOUBLE_PRECISION(precision=53), table=<BourseHistory>, nullable=False), Column('Previous_Value', DOUBLE_PRECISION(precision=53), table=<BourseHistory>, nullable=False), Column('Closed_Day', BOOLEAN(), table=<BourseHistory>, nullable=False), schema=None), 'CryptocurrencyHistory': Table('CryptocurrencyHistory', MetaData(), Column('id', INTEGER(), table=<CryptocurrencyHistory>, primary_key=True, nullable=False, server_default=DefaultClause(<sqlalchemy.sql.elements.TextClause object at 0x7fdb6631d130>, for_update=False

In [15]:
metadata.tables.keys()

dict_keys(['BourseHistory', 'CryptocurrencyHistory', 'CurrencyHistory', 'GoldHistory', 'FundHistory'])

In [11]:
logging.info('Creating Tables ...')
Base.metadata.create_all(engine)

INFO:root:Creating Tables ...


In [6]:
def get_tmc_data_by_url(url):
    headers = { 
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:121.0) Gecko/20100101 Firefox/121.0"
        }

    response = requests.get(url, headers= headers)
    result = response.json()

    return result

def transform_index_data(data, index_type, index_code):
    df = pd.DataFrame(data['indexB2'])

    df['Date_En'] = df['dEven'].astype(str).apply(lambda x: dt.datetime(int(x[:4]), int(x[4:6]), int(x[6:8])))
    df['Index_Id'] = df['insCode']
    df['Value'] = df['xNivInuPbMresIbs']
    df['Previous_Value'] = df['xNivInuClMresIbs']
    df['Index_Type'] = index_type
    df = df[['Index_Id', 'Index_Type', 'Date_En', 'Value', 'Previous_Value']]
    df['Closed_Day'] = False

    time_frame = pd.DataFrame({
        'Date_En': pd.date_range(start='2022-01-01', end=dt.datetime.now()),
        'Index_Id': index_code,
        'Index_Type': index_type
        })

    df = pd.merge(time_frame, df, how= 'left')
    df['Closed_Day'] = df['Closed_Day'].fillna(True)
    df[['Value', 'Previous_Value']] = df[['Value', 'Previous_Value']].ffill()

    return df

index_metadata = {
    'total': 32097828799138957,
    'weighted': 67130298613737946
}

temp = []
for index_type, index_code in index_metadata.items():
    url = f'https://cdn.tsetmc.com/api/Index/GetIndexB2History/{index_code}'
    data = get_tmc_data_by_url(url)
    temp.append(transform_index_data(data, index_type, index_code))
temp = pd.concat(temp)
# temp.to_sql(
#     'IndexHistory',
#     con= engine,
#     if_exists= 'append',
#     index= False
# )

In [7]:
values = temp.to_dict(orient= 'records')

In [8]:
from sqlalchemy import insert
stmt = insert(BourseHistory).values(values)

with engine.connect() as conn:
    result = conn.execute(stmt)
    conn.commit()

In [9]:
from sqlalchemy_utils import create_view
from sqlalchemy import select, func

In [33]:
stmt = select(
    BourseHistory.Index_Type,
    BourseHistory.Value,
    ((BourseHistory.Value - func.lag(BourseHistory.Value).over(order_by= BourseHistory.Date_En)) / func.lag(BourseHistory.Value).over(order_by= BourseHistory.Date_En)).label('daily_rate_of_change'),
    ((BourseHistory.Value - func.lag(BourseHistory.Value, 7).over(order_by= BourseHistory.Date_En)) / func.lag(BourseHistory.Value, 7).over(order_by= BourseHistory.Date_En)).label('weekly_rate_of_change')
    ).where(BourseHistory.Index_Type == "total")

In [36]:
view = 

In [37]:
class Bourse_Total_View(Base):
    __table__ = create_view('Bourse_Total_View', stmt, Base.metadata)

In [39]:
Base.metadata.create_all(engine)

In [ ]:
func.